## Diagnóstico

Raúl Diaz Campos | 1720688J | Sección 3

In [34]:
import json
from pandas import DataFrame as df

In [2]:
FILE = "./farmers-protest-tweets-2021-03-5.json"

In [38]:
def get_tweets(FILE):
    return [json.loads(tweet) for tweet in open(FILE)]

def show_results(results, keys=[]):
    if not keys:
        return df(results)
    return df([{(a if type(a) != list else a[0]) : tweet[a] if type(a) != list else tweet[a[0]][a[1]] for a in keys} for tweet in results])

In [6]:
tweets = get_tweets(FILE)

In [7]:
# Primera función
def retweeted_tweets(tweets):
    def retweet(tweet):
        return tweet["retweetCount"]
    
    most_retweeted = []
    
    for tweet in tweets:
            if len(most_retweeted) <= 9:
                most_retweeted.append(tweet)
                most_retweeted = sorted(most_retweeted, key=retweet)
            else:
                if most_retweeted[9]["retweetCount"] <= tweet["retweetCount"]:
                    most_retweeted.pop(0)
                    most_retweeted.append(tweet)
                elif most_retweeted[0]["retweetCount"] <= tweet["retweetCount"]:
                    most_retweeted.append(tweet)
                    most_retweeted = sorted(most_retweeted, key=retweet)[1:]
    return most_retweeted

In [39]:
show_results(retweeted_tweets(tweets), ["id", "content", ["user", "username"], "retweetCount"])

,id,content,user,retweetCount
0,1359013362881994752,To all of my influencer/celeb friends- read up...,AmandaCerny,15677
1,1357009721090138112,Should be talking about this! #FarmersProtest\...,kylekuzma,17368
2,1356681136655769605,Farmers feed the world. Fight for them. Protec...,vanessa_vash,18744
3,1357755699162398720,There has been much social media coverage arou...,BobBlackman,20132
4,1357048037302960129,"Happy to share that I’ve donated $10,000 to pr...",TeamJuJu,23251
5,1356827705161879553,What in the human rights violations is going o...,miakhalifa,26972
6,1356848397899112448,"“Paid actors,” huh? Quite the casting director...",miakhalifa,35921
7,1357282507616645122,I still #StandWithFarmers and support their pe...,GretaThunberg,67694
8,1356694884615340037,We stand in solidarity with the #FarmersProtes...,GretaThunberg,103957
9,1356625889602199552,why aren’t we talking about this?! #FarmersPro...,rihanna,315547


In [ ]:
# Segunda función
def users_with_more_tweets(tweets):
    pass

In [ ]:
# Tercera función
def days_with_more_tweets(tweets):
    pass

In [ ]:
# Cuarta función
def hashtags_with_more_tweets(tweets):
    pass

In [ ]:
# Main
def main():
    pass